In [1]:
import earthaccess 
import xarray as xr
from virtualizarr import open_virtual_dataset

# To deal with NASA earthdata login :(
import netrc 
import fsspec
import aiohttp

In [2]:
results = earthaccess.search_data(
    short_name = 'M2SDNXSLV',
    temporal = ("2024-01-01", "2024-05-31"))

Granules found: 122


In [3]:
data_links = [granule.data_links(access="external")[0] for granule in results] #access='direct' for in-region AWS S3 us-west-2
data_links[:3]

['https://data.gesdisc.earthdata.nasa.gov/data/MERRA2/M2SDNXSLV.5.12.4/2024/01/MERRA2_400.statD_2d_slv_Nx.20240101.nc4',
 'https://data.gesdisc.earthdata.nasa.gov/data/MERRA2/M2SDNXSLV.5.12.4/2024/01/MERRA2_400.statD_2d_slv_Nx.20240102.nc4',
 'https://data.gesdisc.earthdata.nasa.gov/data/MERRA2/M2SDNXSLV.5.12.4/2024/01/MERRA2_400.statD_2d_slv_Nx.20240103.nc4']

In [4]:
(username, account, password) = netrc.netrc().authenticators("urs.earthdata.nasa.gov")
fsspec.config.conf['https'] = dict(client_kwargs={'auth': aiohttp.BasicAuth(username, password)})

In [12]:
# open file-like object w/ xarray
#ds = xr.open_dataset(fsspec.open(data_links[0]).open(), chunks={})
#ds

<xarray.Dataset> Size: 4MB
Dimensions:     (lon: 576, lat: 361, time: 1)
Coordinates:
  * lon         (lon) float64 5kB -180.0 -179.4 -178.8 ... 178.1 178.8 179.4
  * lat         (lat) float64 3kB -90.0 -89.5 -89.0 -88.5 ... 89.0 89.5 90.0
  * time        (time) datetime64[ns] 8B 2024-01-01T12:00:00
Data variables:
    HOURNORAIN  (time, lat, lon) float32 832kB dask.array<chunksize=(1, 91, 144), meta=np.ndarray>
    T2MMAX      (time, lat, lon) float32 832kB dask.array<chunksize=(1, 91, 144), meta=np.ndarray>
    T2MMEAN     (time, lat, lon) float32 832kB dask.array<chunksize=(1, 91, 144), meta=np.ndarray>
    T2MMIN      (time, lat, lon) float32 832kB dask.array<chunksize=(1, 91, 144), meta=np.ndarray>
    TPRECMAX    (time, lat, lon) float32 832kB dask.array<chunksize=(1, 91, 144), meta=np.ndarray>
Attributes: (12/31)
    NCO:                               netCDF Operators version 4.9.3-alpha05...
    Comment:                           GMAO filename: d5124_m2_jan10.statD_2d...
    History:                           Original file generated: Thu Jan 11 20...
    Filename:                          MERRA2_400.statD_2d_slv_Nx.20240101.nc4
    Conventions:                       CF-1
    Institution:                       NASA Global Modeling and Assimilation ...
    ...                                ...
    Contact:                           http://gmao.gsfc.nasa.gov
    identifier_product_doi:            10.5067/9SC1VNTWGWV3
    RangeBeginningDate:                2024-01-01
    RangeBeginningTime:                00:00:00.000000
    RangeEndingDate:                   2024-01-01
    RangeEndingTime:                   23:59:59.000000

In [13]:

# NOTE: automatically uses preferred chunks when reading (ManifestArray<shape=(361,), dtype=float64, chunks=(91,)>)
# need to decode 'time' https://github.com/zarr-developers/VirtualiZarr/pull/122
vds1 = open_virtual_dataset(data_links[0], indexes={}, loadable_variables='time', filetype='netcdf4',reader_options={})
vds1


<xarray.Dataset> Size: 4MB
Dimensions:     (time: 1, lat: 361, lon: 576)
Coordinates:
    lat         (lat) float64 3kB ManifestArray<shape=(361,), dtype=float64, ...
    lon         (lon) float64 5kB ManifestArray<shape=(576,), dtype=float64, ...
  * time        (time) datetime64[ns] 8B 2024-01-01T12:00:00
Data variables:
    HOURNORAIN  (time, lat, lon) float32 832kB ManifestArray<shape=(1, 361, 5...
    T2MMAX      (time, lat, lon) float32 832kB ManifestArray<shape=(1, 361, 5...
    T2MMEAN     (time, lat, lon) float32 832kB ManifestArray<shape=(1, 361, 5...
    T2MMIN      (time, lat, lon) float32 832kB ManifestArray<shape=(1, 361, 5...
    TPRECMAX    (time, lat, lon) float32 832kB ManifestArray<shape=(1, 361, 5...
Attributes: (12/31)
    Comment:                           GMAO filename: d5124_m2_jan10.statD_2d...
    Contact:                           http://gmao.gsfc.nasa.gov
    Conventions:                       CF-1
    DataResolution:                    0.5 x 0.625
    EasternmostLongitude:              179.375
    Filename:                          MERRA2_400.statD_2d_slv_Nx.20240101.nc4
    ...                                ...
    TemporalRange:                     1980-01-01 -> 2016-12-31
    Title:                             MERRA2 statD_2d_slv_Nx: 2d,Daily,Aggre...
    VersionID:                         5.12.4
    WesternmostLongitude:              -180.0
    identifier_product_doi:            10.5067/9SC1VNTWGWV3
    identifier_product_doi_authority:  http://dx.doi.org/

In [14]:
vds1[['TPRECMAX']].virtualize.to_kerchunk('trpecmax-remote.json', format='json')